In [1]:
import os
import glob
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pymysql

In [8]:
import os
import glob
import re
from pathlib import Path

import pandas as pd
import numpy as np
import pymysql


# =========================
# ✅ 설정(여기만 수정)
# =========================
ROOT_DIR = r"C:\Users\user\Documents\서울시_포폴\서울시유동인구_스냅샷"  # 2023/2024/2025 폴더가 있는 상위 경로

MYSQL = dict(
    host="localhost",
    user="root",
    password="********",
    db="seoul",
    port=3306,
    charset="utf8mb4",
    autocommit=False
)

TABLE = "stg_metro_people_snapshot"


# =========================
# 내부 유틸
# =========================
def read_csv_with_encoding(path: str) -> pd.DataFrame:
    """서울시 CSV에서 흔한 인코딩을 순서대로 시도"""
    for enc in ("utf-8-sig", "cp949", "euc-kr"):
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            continue
    raise ValueError(f"인코딩 실패: {path}")


def clean_columns(df: pd.DataFrame) -> pd.DataFrame:
    """컬럼명에서 ?, " 제거 + 공백 제거"""
    df = df.copy()
    df.columns = (df.columns.astype(str)
                  .str.replace('"', '', regex=False)
                  .str.replace('?', '', regex=False)
                  .str.strip())
    return df


def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    """* -> NULL, 숫자형 변환, 기준일 생성"""
    df = df.copy()

    # 비식별 처리(*) -> NaN
    df = df.replace('*', np.nan)

    # 기준일ID -> 기준일(DATE)
    if "기준일ID" in df.columns:
        df["기준일"] = pd.to_datetime(df["기준일ID"].astype(str), format="%Y%m%d", errors="coerce")
    else:
        df["기준일"] = pd.NaT

    # 숫자형 변환(키 컬럼 제외)
    key_cols = ["기준일ID", "시간대구분", "행정동코드", "대도시권거주지코드"]
    for c in df.columns:
        if c in key_cols or c == "기준일":
            continue
        df[c] = pd.to_numeric(df[c], errors="coerce")

    return df


def iter_snapshot_files(root_dir: str):
    """2023/2024/2025 하위의 METRO_PEOPLE_*.csv 전부 순회"""
    root = Path(root_dir)
    for year_dir in sorted([p for p in root.iterdir() if p.is_dir() and p.name.isdigit()]):
        # 파일명이 METRO_PEOPLE_YYYYMMDD.csv 형태라고 했으니 이 패턴만 우선
        for path in sorted(year_dir.glob("METRO_PEOPLE_*.csv")):
            yield str(path)


def get_insert_sql(cols):
    col_sql = ",".join([f"`{c}`" for c in cols])
    placeholders = ",".join(["%s"] * len(cols))

    # 중복키(기준일ID,시간대구분,행정동코드,대도시권거주지코드)는 업데이트 제외
    key_set = {"기준일ID", "시간대구분", "행정동코드", "대도시권거주지코드"}
    update_cols = [c for c in cols if c not in key_set]
    update_sql = ",".join([f"`{c}`=VALUES(`{c}`)" for c in update_cols])

    sql = f"""
    INSERT INTO {TABLE} ({col_sql})
    VALUES ({placeholders})
    ON DUPLICATE KEY UPDATE
    기준일ID = 기준일ID
    """
    return sql


def load_one_file(conn, path: str, chunk_size=10000):
    df = read_csv_with_encoding(path)
    df = clean_columns(df)

    # ✅ 키 컬럼 존재 확인
    required = {"기준일ID", "시간대구분", "행정동코드", "대도시권거주지코드"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"필수 컬럼 누락 {missing} in {path}")

    # =========================
    # 1️⃣ 전처리
    # =========================
    df = preprocess(df)

    # =========================
    # 2️⃣ 🔒 유니크키 컬럼 NULL 방지 (여기에!)
    # =========================
    df = df.dropna(subset=[
        "기준일ID",
        "시간대구분",
        "행정동코드",
        "대도시권거주지코드"
    ])

    # =========================
    # 3️⃣ 메타 정보 추가 (그 다음!)
    # =========================
    df["source_file"] = os.path.basename(path)

    # =========================
    # 4️⃣ DB 컬럼 순서 정의
    # =========================
    cols = [
        "기준일ID","기준일","시간대구분","행정동코드","대도시권거주지코드",
        "생활인구순위","총생활인구수",
        "남자10세부터14세생활인구수","남자15세부터19세생활인구수","남자20세부터24세생활인구수",
        "남자25세부터29세생활인구수","남자30세부터34세생활인구수","남자35세부터39세생활인구수",
        "남자40세부터44세생활인구수","남자45세부터49세생활인구수","남자50세부터54세생활인구수",
        "남자55세부터59세생활인구수","남자60세부터64세생활인구수","남자65세부터69세생활인구수",
        "남자70세부터74세생활인구수","남자75세부터79세생활인구수",
        "여자10세부터14세생활인구수","여자15세부터19세생활인구수","여자20세부터24세생활인구수",
        "여자25세부터29세생활인구수","여자30세부터34세생활인구수","여자35세부터39세생활인구수",
        "여자40세부터44세생활인구수","여자45세부터49세생활인구수","여자50세부터54세생활인구수",
        "여자55세부터59세생활인구수","여자60세부터64세생활인구수","여자65세부터69세생활인구수",
        "여자70세부터74세생활인구수","여자75세부터79세생활인구수",
        "장기체류외국인수",
        "source_file"
    ]

    # =========================
    # 5️⃣ DATE 컬럼 문자열화
    # =========================
    df["기준일"] = df["기준일"].dt.strftime("%Y-%m-%d")

    # =========================
    # 6️⃣ NaN/NA/NaT → None (pymysql 안전)
    # =========================
    sub = df[cols].copy()
    sub = sub.astype(object).where(pd.notnull(sub), None)
    data = [tuple(x) for x in sub.to_numpy()]

    sql = get_insert_sql(cols)

    with conn.cursor() as cur:
        for i in range(0, len(data), chunk_size):
            cur.executemany(sql, data[i:i+chunk_size])
            
    conn.commit()   # ✅ 파일 하나 끝나면 커밋
    
    return len(df)


def main():
    conn = pymysql.connect(**MYSQL)

    ok, fail = 0, 0
    total_rows = 0

    for path in iter_snapshot_files(ROOT_DIR):
        try:
            n = load_one_file(conn, path)
            ok += 1
            total_rows += n
            print(f"[OK] {os.path.basename(path)}  rows={n}")
        except Exception as e:
            fail += 1
            print(f"[FAIL] {path}  -> {e}")

    conn.close()
    print("=====================================")
    print(f"files ok={ok}, fail={fail}, total_rows_inserted={total_rows}")
    print("done")

if __name__ == "__main__":
    main()


KeyboardInterrupt: 

In [9]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import pymysql


# =========================
# 🔧 설정 (여기만 수정)
# =========================
ROOT_DIR = r"C:\Users\user\Documents\서울시_포폴\서울시유동인구_스냅샷"

MYSQL = dict(
    host="localhost",
    user="root",
    password="Abcd1234!",
    db="seoul",
    port=3306,
    charset="utf8mb4",
    autocommit=False   # ⭐ 중요
)

TABLE = "stg_metro_people_snapshot"
CHUNK_SIZE = 20000


# =========================
# 유틸 함수
# =========================
def read_csv_auto_encoding(path):
    for enc in ("utf-8-sig", "cp949", "euc-kr"):
        try:
            return pd.read_csv(path, encoding=enc)
        except Exception:
            pass
    raise ValueError(f"인코딩 실패: {path}")


def clean_columns(df):
    df.columns = (
        df.columns.astype(str)
        .str.replace('"', '', regex=False)
        .str.replace('?', '', regex=False)
        .str.strip()
    )
    return df


def preprocess(df):
    # * → NaN
    df = df.replace('*', np.nan)

    # 기준일
    df["기준일"] = pd.to_datetime(
        df["기준일ID"].astype(str),
        format="%Y%m%d",
        errors="coerce"
    )

    # 숫자형 변환
    key_cols = ["기준일ID", "시간대구분", "행정동코드", "대도시권거주지코드"]
    for c in df.columns:
        if c not in key_cols + ["기준일"]:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    return df


def iter_files(root):
    root = Path(root)
    for y in sorted(p for p in root.iterdir() if p.is_dir()):
        for f in sorted(y.glob("METRO_PEOPLE_*.csv")):
            yield str(f)


def get_insert_sql(cols):
    col_sql = ",".join(f"`{c}`" for c in cols)
    placeholders = ",".join(["%s"] * len(cols))
    return f"INSERT IGNORE INTO {TABLE} ({col_sql}) VALUES ({placeholders})"


# =========================
# 핵심 로더
# =========================
def load_one_file(conn, path):
    print(f"\n[START] {os.path.basename(path)}")

    df = read_csv_auto_encoding(path)
    df = clean_columns(df)

    # 필수 키 체크
    required = {"기준일ID", "시간대구분", "행정동코드", "대도시권거주지코드"}
    if not required.issubset(df.columns):
        raise ValueError(f"필수 컬럼 누락: {required - set(df.columns)}")

    df = preprocess(df)

    # 🔒 유니크키 NULL 방지
    df = df.dropna(subset=[
        "기준일ID", "시간대구분", "행정동코드", "대도시권거주지코드"
    ])

    df["source_file"] = os.path.basename(path)
    df["기준일"] = df["기준일"].dt.strftime("%Y-%m-%d")

    cols = [
        "기준일ID","기준일","시간대구분","행정동코드","대도시권거주지코드",
        "생활인구순위","총생활인구수",
        "남자10세부터14세생활인구수","남자15세부터19세생활인구수","남자20세부터24세생활인구수",
        "남자25세부터29세생활인구수","남자30세부터34세생활인구수","남자35세부터39세생활인구수",
        "남자40세부터44세생활인구수","남자45세부터49세생활인구수","남자50세부터54세생활인구수",
        "남자55세부터59세생활인구수","남자60세부터64세생활인구수","남자65세부터69세생활인구수",
        "남자70세부터74세생활인구수","남자75세부터79세생활인구수",
        "여자10세부터14세생활인구수","여자15세부터19세생활인구수","여자20세부터24세생활인구수",
        "여자25세부터29세생활인구수","여자30세부터34세생활인구수","여자35세부터39세생활인구수",
        "여자40세부터44세생활인구수","여자45세부터49세생활인구수","여자50세부터54세생활인구수",
        "여자55세부터59세생활인구수","여자60세부터64세생활인구수","여자65세부터69세생활인구수",
        "여자70세부터74세생활인구수","여자75세부터79세생활인구수",
        "장기체류외국인수",
        "source_file"
    ]

    # NaN/NA/NaT → None
    sub = df[cols].astype(object).where(pd.notnull(df[cols]), None)
    data = [tuple(r) for r in sub.to_numpy()]

    sql = get_insert_sql(cols)

    with conn.cursor() as cur:
        for i in range(0, len(data), CHUNK_SIZE):
            cur.executemany(sql, data[i:i+CHUNK_SIZE])
            print(f"  inserted {min(i+CHUNK_SIZE, len(data))}/{len(data)}")

    conn.commit()
    print(f"[DONE] {os.path.basename(path)} rows={len(df)}")


# =========================
# main
# =========================
def main():
    conn = pymysql.connect(**MYSQL)

    ok, fail = 0, 0
    for path in iter_files(ROOT_DIR):
        try:
            load_one_file(conn, path)
            ok += 1
        except Exception as e:
            fail += 1
            print(f"[FAIL] {path} -> {e}")

    conn.close()
    print("\n===================================")
    print(f"files ok={ok}, fail={fail}")
    print("ALL DONE")


if __name__ == "__main__":
    main()



[START] METRO_PEOPLE_20230116.csv
  inserted 20000/591407
  inserted 40000/591407
  inserted 60000/591407
  inserted 80000/591407
  inserted 100000/591407
  inserted 120000/591407
  inserted 140000/591407
  inserted 160000/591407
  inserted 180000/591407
  inserted 200000/591407
  inserted 220000/591407
  inserted 240000/591407
  inserted 260000/591407
  inserted 280000/591407
  inserted 300000/591407
  inserted 320000/591407
  inserted 340000/591407
  inserted 360000/591407
  inserted 380000/591407
  inserted 400000/591407
  inserted 420000/591407
  inserted 440000/591407
  inserted 460000/591407
  inserted 480000/591407
  inserted 500000/591407
  inserted 520000/591407
  inserted 540000/591407
  inserted 560000/591407
  inserted 580000/591407
  inserted 591407/591407
[DONE] METRO_PEOPLE_20230116.csv rows=591407

[START] METRO_PEOPLE_20230215.csv
  inserted 20000/586627
  inserted 40000/586627
  inserted 60000/586627
  inserted 80000/586627
  inserted 100000/586627
  inserted 120000/